# Airline Delays Data Analysis (Python Refresh)

This notebook modernizes the original 2021 R analysis using weighted rate metrics, severity metrics, airport profiling, cause-composition analysis, and baseline predictive modeling.

## 1) Setup and data loading

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

sns.set_theme(style='whitegrid', context='talk')
pd.set_option('display.max_columns', 100)

DATA_PATH = 'airlines.csv'
df = pd.read_csv(DATA_PATH)
print('Shape:', df.shape)
display(df.head(3))

In [ ]:
# Normalize month label typo for cleaner charts
df['Time.Month Name'] = df['Time.Month Name'].replace({'Febuary': 'February'})

# Quick quality checks
assert df.isna().sum().sum() == 0, 'Unexpected missing values found.'

flight_balance = (
    df['Statistics.Flights.Cancelled']
    + df['Statistics.Flights.Delayed']
    + df['Statistics.Flights.Diverted']
    + df['Statistics.Flights.On Time']
)
assert (flight_balance == df['Statistics.Flights.Total']).all(), 'Flight totals are not balanced.'

print('Year range:', df['Time.Year'].min(), '-', df['Time.Year'].max())
print('Airports:', df['Airport.Code'].nunique())

## 2) Weighted KPI framework (recommended update)

The old workflow often averaged row-level counts directly. Here we use weighted rates: aggregate raw counts first, then divide by total flights.

In [ ]:
def add_rate_columns(frame):
    out = frame.copy()
    out['on_time_rate'] = out['Flights_On_Time'] / out['Flights_Total']
    out['delay_rate'] = out['Flights_Delayed'] / out['Flights_Total']
    out['cancel_rate'] = out['Flights_Cancelled'] / out['Flights_Total']
    out['divert_rate'] = out['Flights_Diverted'] / out['Flights_Total']
    return out

yearly = (
    df.groupby('Time.Year', as_index=False)
      .agg(
          Flights_Total=('Statistics.Flights.Total', 'sum'),
          Flights_On_Time=('Statistics.Flights.On Time', 'sum'),
          Flights_Delayed=('Statistics.Flights.Delayed', 'sum'),
          Flights_Cancelled=('Statistics.Flights.Cancelled', 'sum'),
          Flights_Diverted=('Statistics.Flights.Diverted', 'sum')
      )
)
yearly = add_rate_columns(yearly)
display(yearly.head())

best_year = yearly.loc[yearly['on_time_rate'].idxmax(), ['Time.Year', 'on_time_rate']]
worst_year = yearly.loc[yearly['on_time_rate'].idxmin(), ['Time.Year', 'on_time_rate']]
print('Best on-time year:', int(best_year['Time.Year']), f"({best_year['on_time_rate']:.2%})")
print('Worst on-time year:', int(worst_year['Time.Year']), f"({worst_year['on_time_rate']:.2%})")

In [ ]:
kpi_long = yearly.melt(
    id_vars='Time.Year',
    value_vars=['on_time_rate', 'delay_rate', 'cancel_rate', 'divert_rate'],
    var_name='metric',
    value_name='value'
)

plt.figure(figsize=(13, 6))
sns.lineplot(data=kpi_long, x='Time.Year', y='value', hue='metric', marker='o')
plt.gca().yaxis.set_major_formatter(plt.matplotlib.ticker.PercentFormatter(1.0))
plt.title('Weighted Flight Outcome Rates by Year')
plt.xlabel('Year')
plt.ylabel('Rate')
plt.legend(title='KPI', bbox_to_anchor=(1.02, 1), loc='upper left')
plt.tight_layout()
plt.show()

## 3) Delay-cause composition and drift

In [ ]:
count_reason_map = {
    'Carrier': 'Statistics.# of Delays.Carrier',
    'Late Aircraft': 'Statistics.# of Delays.Late Aircraft',
    'NAS': 'Statistics.# of Delays.National Aviation System',
    'Security': 'Statistics.# of Delays.Security',
    'Weather': 'Statistics.# of Delays.Weather',
}

minutes_reason_map = {
    'Carrier': 'Statistics.Minutes Delayed.Carrier',
    'Late Aircraft': 'Statistics.Minutes Delayed.Late Aircraft',
    'NAS': 'Statistics.Minutes Delayed.National Aviation System',
    'Security': 'Statistics.Minutes Delayed.Security',
    'Weather': 'Statistics.Minutes Delayed.Weather',
}

reason_totals = pd.DataFrame({
    'reason': list(count_reason_map.keys()),
    'delay_count': [df[col].sum() for col in count_reason_map.values()],
    'delay_minutes': [df[col].sum() for col in minutes_reason_map.values()]
})
reason_totals['share_by_count'] = reason_totals['delay_count'] / reason_totals['delay_count'].sum()
reason_totals['share_by_minutes'] = reason_totals['delay_minutes'] / reason_totals['delay_minutes'].sum()
reason_totals = reason_totals.sort_values('share_by_count', ascending=True)
display(reason_totals)

fig, axes = plt.subplots(1, 2, figsize=(15, 6), sharey=True)
sns.barplot(data=reason_totals, x='share_by_count', y='reason', ax=axes[0], color='#4C72B0')
axes[0].set_title('Delay Cause Share (Count)')
axes[0].xaxis.set_major_formatter(plt.matplotlib.ticker.PercentFormatter(1.0))
axes[0].set_xlabel('Share')
axes[0].set_ylabel('Reason')

sns.barplot(data=reason_totals, x='share_by_minutes', y='reason', ax=axes[1], color='#55A868')
axes[1].set_title('Delay Cause Share (Minutes)')
axes[1].xaxis.set_major_formatter(plt.matplotlib.ticker.PercentFormatter(1.0))
axes[1].set_xlabel('Share')
axes[1].set_ylabel('')

plt.tight_layout()
plt.show()

In [ ]:
year_reason = (
    df.groupby('Time.Year', as_index=False)[list(count_reason_map.values())]
      .sum()
)
year_reason_long = year_reason.melt(
    id_vars='Time.Year',
    var_name='reason_col',
    value_name='delay_count'
)
inv_count = {v: k for k, v in count_reason_map.items()}
year_reason_long['reason'] = year_reason_long['reason_col'].map(inv_count)
year_reason_long['share'] = year_reason_long['delay_count'] / year_reason_long.groupby('Time.Year')['delay_count'].transform('sum')

pivot_share = year_reason_long.pivot(index='Time.Year', columns='reason', values='share').sort_index()
pivot_share.plot.area(figsize=(13, 6), colormap='tab20')
plt.title('Delay Cause Composition Drift Over Time (by Count Share)')
plt.ylabel('Share of delayed flights')
plt.gca().yaxis.set_major_formatter(plt.matplotlib.ticker.PercentFormatter(1.0))
plt.xlabel('Year')
plt.legend(title='Reason', bbox_to_anchor=(1.02, 1), loc='upper left')
plt.tight_layout()
plt.show()

## 4) Seasonality and monthly heatmap

In [ ]:
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']

monthly = (
    df.groupby(['Time.Year', 'Time.Month Name'], as_index=False)
      .agg(
          Flights_Total=('Statistics.Flights.Total', 'sum'),
          Flights_Delayed=('Statistics.Flights.Delayed', 'sum')
      )
)
monthly['delay_rate'] = monthly['Flights_Delayed'] / monthly['Flights_Total']
monthly['Time.Month Name'] = pd.Categorical(monthly['Time.Month Name'], categories=month_order, ordered=True)

heat = monthly.pivot(index='Time.Month Name', columns='Time.Year', values='delay_rate').loc[month_order]
plt.figure(figsize=(14, 7))
sns.heatmap(heat, cmap='YlOrRd', cbar_kws={'format': plt.matplotlib.ticker.PercentFormatter(1.0)})
plt.title('Delay Rate Seasonality Heatmap (Month x Year)')
plt.xlabel('Year')
plt.ylabel('Month')
plt.tight_layout()
plt.show()

## 5) Severity metrics (minutes per delayed flight)

In [ ]:
eps = 1e-9

severity_year = (
    df.groupby('Time.Year', as_index=False)
      .agg(
          delay_minutes=('Statistics.Minutes Delayed.Total', 'sum'),
          delayed_flights=('Statistics.Flights.Delayed', 'sum')
      )
)
severity_year['avg_delay_minutes_per_delayed_flight'] = severity_year['delay_minutes'] / (severity_year['delayed_flights'] + eps)
display(severity_year)

plt.figure(figsize=(12, 5))
sns.lineplot(data=severity_year, x='Time.Year', y='avg_delay_minutes_per_delayed_flight', marker='o')
plt.title('Average Delay Minutes per Delayed Flight (Yearly)')
plt.xlabel('Year')
plt.ylabel('Minutes')
plt.tight_layout()
plt.show()

In [ ]:
reason_severity = pd.DataFrame({'reason': list(count_reason_map.keys())})
reason_severity['delays'] = [df[count_reason_map[r]].sum() for r in reason_severity['reason']]
reason_severity['minutes'] = [df[minutes_reason_map[r]].sum() for r in reason_severity['reason']]
reason_severity['minutes_per_delay'] = reason_severity['minutes'] / (reason_severity['delays'] + eps)
reason_severity = reason_severity.sort_values('minutes_per_delay', ascending=False)
display(reason_severity)

plt.figure(figsize=(10, 5))
sns.barplot(data=reason_severity, x='minutes_per_delay', y='reason', color='#C44E52')
plt.title('Severity by Reason (Minutes per Delay)')
plt.xlabel('Minutes per delayed flight event')
plt.ylabel('Reason')
plt.tight_layout()
plt.show()

## 6) Airport profiles and volatility

In [ ]:
airport_year = (
    df.groupby(['Airport.Code', 'Airport.Name', 'Time.Year'], as_index=False)
      .agg(
          Flights_Total=('Statistics.Flights.Total', 'sum'),
          Flights_On_Time=('Statistics.Flights.On Time', 'sum'),
          Flights_Delayed=('Statistics.Flights.Delayed', 'sum'),
          delay_minutes=('Statistics.Minutes Delayed.Total', 'sum')
      )
)
airport_year['on_time_rate'] = airport_year['Flights_On_Time'] / airport_year['Flights_Total']
airport_year['delay_rate'] = airport_year['Flights_Delayed'] / airport_year['Flights_Total']
airport_year['severity_minutes_per_delayed'] = airport_year['delay_minutes'] / (airport_year['Flights_Delayed'] + eps)

airport_profile = (
    airport_year.groupby(['Airport.Code', 'Airport.Name'], as_index=False)
               .agg(
                   total_flights=('Flights_Total', 'sum'),
                   long_run_on_time_rate=('on_time_rate', 'mean'),
                   long_run_delay_rate=('delay_rate', 'mean'),
                   annual_delay_rate_volatility=('delay_rate', 'std'),
                   avg_severity=('severity_minutes_per_delayed', 'mean')
               )
)
airport_profile = airport_profile.sort_values('total_flights', ascending=False)
display(airport_profile.head(10))

In [ ]:
# Filter to busiest airports for fair comparison
threshold = airport_profile['total_flights'].quantile(0.50)
airport_filtered = airport_profile[airport_profile['total_flights'] >= threshold].copy()

top_reliable = airport_filtered.sort_values('long_run_on_time_rate', ascending=False).head(10)
top_volatile = airport_filtered.sort_values('annual_delay_rate_volatility', ascending=False).head(10)

display(top_reliable[['Airport.Code', 'long_run_on_time_rate', 'total_flights']])
display(top_volatile[['Airport.Code', 'annual_delay_rate_volatility', 'total_flights']])

plt.figure(figsize=(12, 8))
sns.scatterplot(
    data=airport_filtered,
    x='long_run_delay_rate',
    y='avg_severity',
    size='total_flights',
    hue='annual_delay_rate_volatility',
    palette='viridis',
    alpha=0.8
)
for _, r in airport_filtered.nlargest(8, 'total_flights').iterrows():
    plt.text(r['long_run_delay_rate'], r['avg_severity'], r['Airport.Code'], fontsize=9)
plt.gca().xaxis.set_major_formatter(plt.matplotlib.ticker.PercentFormatter(1.0))
plt.title('Airport Risk Map: Frequency vs Severity')
plt.xlabel('Long-run delay rate')
plt.ylabel('Average severity (minutes per delayed flight)')
plt.tight_layout()
plt.show()

## 7) Baseline predictive modeling (time-aware split)

In [ ]:
model_df = (
    df.groupby(['Airport.Code', 'Time.Year', 'Time.Month'], as_index=False)
      .agg(
          carriers_total=('Statistics.Carriers.Total', 'mean'),
          flights_total=('Statistics.Flights.Total', 'sum'),
          flights_delayed=('Statistics.Flights.Delayed', 'sum'),
          delay_minutes_total=('Statistics.Minutes Delayed.Total', 'sum'),
          delay_carrier=('Statistics.# of Delays.Carrier', 'sum'),
          delay_late_aircraft=('Statistics.# of Delays.Late Aircraft', 'sum'),
          delay_nas=('Statistics.# of Delays.National Aviation System', 'sum'),
          delay_security=('Statistics.# of Delays.Security', 'sum'),
          delay_weather=('Statistics.# of Delays.Weather', 'sum')
      )
)
model_df['delay_rate'] = model_df['flights_delayed'] / (model_df['flights_total'] + eps)
model_df['severity'] = model_df['delay_minutes_total'] / (model_df['flights_delayed'] + eps)

reason_cols = ['delay_carrier', 'delay_late_aircraft', 'delay_nas', 'delay_security', 'delay_weather']
reason_sum = model_df[reason_cols].sum(axis=1) + eps
for c in reason_cols:
    model_df[f'{c}_share'] = model_df[c] / reason_sum

# Cyclical month encoding
model_df['month_sin'] = np.sin(2 * np.pi * model_df['Time.Month'] / 12.0)
model_df['month_cos'] = np.cos(2 * np.pi * model_df['Time.Month'] / 12.0)

# Lag feature: previous month delay rate per airport
model_df = model_df.sort_values(['Airport.Code', 'Time.Year', 'Time.Month']).reset_index(drop=True)
model_df['lag1_delay_rate'] = model_df.groupby('Airport.Code')['delay_rate'].shift(1)

display(model_df.head())

In [ ]:
feature_cols_num = [
    'Time.Year', 'carriers_total', 'flights_total', 'month_sin', 'month_cos',
    'lag1_delay_rate',
    'delay_carrier_share', 'delay_late_aircraft_share', 'delay_nas_share',
    'delay_security_share', 'delay_weather_share'
]
feature_cols_cat = ['Airport.Code']

train = model_df[model_df['Time.Year'] <= 2013].copy()
test = model_df[model_df['Time.Year'] >= 2014].copy()

def build_preprocessor():
    num_pipe = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    cat_pipe = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])
    return ColumnTransformer(
        transformers=[
            ('num', num_pipe, feature_cols_num),
            ('cat', cat_pipe, feature_cols_cat)
        ]
    )

def evaluate_regression(target_col):
    X_train = train[feature_cols_num + feature_cols_cat]
    y_train = train[target_col]
    X_test = test[feature_cols_num + feature_cols_cat]
    y_test = test[target_col]

    enet = Pipeline(steps=[
        ('prep', build_preprocessor()),
        ('model', ElasticNet(alpha=0.01, l1_ratio=0.2, random_state=42, max_iter=5000))
    ])
    hgb = Pipeline(steps=[
        ('prep', build_preprocessor()),
        ('model', HistGradientBoostingRegressor(random_state=42))
    ])

    metrics = []
    for name, pipe in [('ElasticNet', enet), ('HistGradientBoosting', hgb)]:
        pipe.fit(X_train, y_train)
        pred = pipe.predict(X_test)
        mae = mean_absolute_error(y_test, pred)
        rmse = np.sqrt(mean_squared_error(y_test, pred))
        metrics.append({'target': target_col, 'model': name, 'MAE': mae, 'RMSE': rmse})

    return pd.DataFrame(metrics)

results_delay = evaluate_regression('delay_rate')
results_severity = evaluate_regression('severity')
display(pd.concat([results_delay, results_severity], ignore_index=True))

## 8) Suggested next improvements

- Add confidence intervals on airport rankings using bootstrap.
- Add external weather signals for better causal interpretation.
- Build an interactive dashboard (Plotly/Altair) with airport and time filters.
- Export clean tables for downstream BI tools.